In [36]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Importing the tokenizer and model.
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

# Model parameters.
param = sum(p.numel() for p in model.parameters())
print("Total Parameters: ", param)

Total Parameters:  76672000


In [27]:
# Creating the inputs for the model with the help of the tokenizer. 
input_ids = tokenizer("Just an example", add_special_tokens=True, return_tensors="pt").input_ids
attn_mask = tokenizer("Just an example", add_special_tokens=True, return_tensors="pt").attention_mask
decoder_input_ids = tokenizer("example continued", add_special_tokens=True, return_tensors="pt").input_ids

print("Input_ids:", input_ids)
print("Attention_mask", attn_mask)
print("Decoder_input_ids", decoder_input_ids)

Input_ids: tensor([[ 2005, 29453,    21,  1965,  4556,    68, 12346,  2064,     0]])
Attention_mask tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])
Decoder_input_ids tensor([[ 4556,    68, 12346,  2064,   726, 26321,   599,    91,   245,     0]])


In [28]:
# Storing the pytorch model's output in pt_outputs variable.
pt_outputs = model(input_ids, attn_mask, decoder_input_ids)

print("Pytorch's Output: ", pt_outputs)

Pytorch's Output:  Seq2SeqLMOutput(loss=None, logits=tensor([[[ 8.2725, -1.6409,  6.5839,  ..., -1.5181, -1.4908,  0.0000],
         [ 7.6752, -2.7486,  6.4476,  ..., -2.6328, -2.6111,  0.0000],
         [ 8.2976, -2.2287,  6.2131,  ..., -2.0858, -2.0529,  0.0000],
         ...,
         [ 6.8837, -3.2134,  3.9941,  ..., -3.0988, -3.0865,  0.0000],
         [ 9.3504, -1.2584,  6.6609,  ..., -1.1112, -1.0927,  0.0000],
         [ 7.8349, -3.0918,  5.2031,  ..., -2.9667, -2.9505,  0.0000]]],
       grad_fn=<AddBackward0>), past_key_values=((tensor([[[[-5.0263e-01, -5.7339e-01,  5.5577e-01,  ...,  2.2565e+00,
            3.0292e+00,  3.0174e+00],
          [ 1.6520e+00, -1.9985e+00,  2.0905e+00,  ...,  1.3953e+00,
            2.0053e-01,  2.5690e+00],
          [ 1.7360e+00,  7.8089e-01,  1.2876e+00,  ...,  3.1986e-01,
            6.8844e-01,  5.3964e-01],
          ...,
          [-2.2641e+00, -5.9427e-01, -3.5047e-01,  ..., -4.5024e-01,
            1.7368e+00,  2.1847e+00],
          [-

In [29]:
# Exporting torch model to onnx by using ONNX_ATEN_FALLBACK this allows
# us to convert this model to onnx but then for the silu activation
# function it fallbacks to ATEN types which are not supported by
# openvino and so the onnx to openvino conversion fails. 
torch.onnx.export(model,
                 (input_ids, attn_mask, decoder_input_ids),
                 "opus-ru-en.onnx",
                 input_names=["input_ids", "attn_mask", "decoder_input_ids"],
                 output_names=["outputs"],
                 opset_version=12,
                 operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK)

In [30]:
# Converting from onnx to openvino fails because of the above mentioned reason.  
!python3 /opt/intel/openvino_2021.2.200/deployment_tools/model_optimizer/mo.py --input_model opus-ru-en.onnx

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/aakash/box_of_ai_tools/Natural_Language_Processing/Machine_Translation/t5-large/opus-ru-en.onnx
	- Path for generated IR: 	/home/aakash/box_of_ai_tools/Natural_Language_Processing/Machine_Translation/t5-large/.
	- IR output name: 	opus-ru-en
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	None
	- Reverse input channels: 	False
ONNX specific parameters:
Model Optimizer version: 	2021.2.0-1877-176bdf51370-releases/2021/2
2021-02-18 23:10:28.200061: W tensorflow/stream_executor/platfo